### 1. Load the data

In [1]:
! wget https://github.com/erkansirin78/datasets/raw/master/market1mil.csv.gz

--2023-04-25 09:20:28--  https://github.com/erkansirin78/datasets/raw/master/market1mil.csv.gz
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erkansirin78/datasets/master/market1mil.csv.gz [following]
--2023-04-25 09:20:28--  https://raw.githubusercontent.com/erkansirin78/datasets/master/market1mil.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44525776 (42M) [application/octet-stream]
Saving to: ‘market1mil.csv.gz’

100%[======================================>] 44,525,776  2.35MB/s   in 31s    

2023-04-25 09:20:59 (1.39 MB/s) - ‘market1mil.csv.gz’ saved [44525776/44525776]



In [2]:
!gzip -d market1mil.csv.gz

### 2.Spark and Elasticsearch Library and Configuration

In [3]:
import findspark
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
from elasticsearch import Elasticsearch, helpers
import time
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *

In [4]:
findspark.init("/opt/manual/spark")

In [5]:
spark = (
    SparkSession.builder
    .appName("Spark Elasticsearch")
    .master("local[2]")
    .config("spark.driver.memory","2048m")
    .config("spark.sql.shuffle.partitions", 4)
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.jars.packages", "org.elasticsearch:elasticsearch-spark-30_2.12:7.12.1") 
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/manual/spark-3.1.1-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/train/.ivy2/cache
The jars for the packages stored in: /home/train/.ivy2/jars
org.elasticsearch#elasticsearch-spark-30_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1ecfd205-29cc-47c6-a409-3c9c0b88ba0a;1.0
	confs: [default]
	found org.elasticsearch#elasticsearch-spark-30_2.12;7.12.1 in central
	found org.scala-lang#scala-reflect;2.12.8 in central
	found org.slf4j#slf4j-api;1.7.6 in local-m2-cache
	found commons-logging#commons-logging;1.1.1 in local-m2-cache
	found javax.xml.bind#jaxb-api;2.3.1 in central
	found com.google.protobuf#protobuf-java;2.5.0 in local-m2-cache
	found org.apache.spark#spark-yarn_2.12;3.0.1 in central
:: resolution report :: resolve 741ms :: artifacts dl 8ms
	:: modules in use:
	com.google.protobuf#protobuf-java;2.5.0 from local-m2-cache in [default]
	commons-logging#commons-logging;1.1.1 from local-m2-cache in [default]
	javax.xml.bind#jaxb-api;2.3.1 from central in [default]
	org.ap

### 3. Read Data

In [6]:
df = spark.read.format("csv") \
          .option("header", True) \
          .option("inferSchema", True) \
          .option("sep", ";") \
          .load("file:////home/train/dataops_homework/7_Week/market1mil.csv") \
          .select("LOGICALREF", "ITEMCODE", "ITEMNAME", "AMOUNT", "PRICE", "LINENETTOTAL", "BRANCH", "CITY", "BRAND", "STARTDATE", "LONGITUDE","LATITUDE") \
          .cache()

In [7]:
df.limit(5).toPandas()

,LOGICALREF,ITEMCODE,ITEMNAME,AMOUNT,PRICE,LINENETTOTAL,BRANCH,CITY,BRAND,STARTDATE,LONGITUDE,LATITUDE
0,1,8,TOZ SEKER,45,"2,65","5,3",Batman Subesi,Batman,None,3.01.2017 09:25,"41,1351","37,8812"
1,2,20868,KIRMIZI MERCIMEK,"1,006","2,8","2,82",Batman Subesi,Batman,BAKLİYAT,3.01.2017 09:25,"41,1351","37,8812"
2,3,8583,"TEST MATIK 1,5 KG NORMAL",1,"4,95","4,95",Batman Subesi,Batman,TEST,3.01.2017 09:25,"41,1351","37,8812"
3,4,1454,BIZIM MAKARNA BONCUK,1,"1,1","1,1",Batman Subesi,Batman,ÜLKER,3.01.2017 09:25,"41,1351","37,8812"
4,5,13519,FILIZ MAKARNA KISA KESME 500 GR,1,"1,1","1,1",Batman Subesi,Batman,FİLİZ,3.01.2017 09:25,"41,1351","37,8812"


In [8]:
df.count()

999853

In [9]:
df1 = df.na.drop()

In [10]:
df1.count()

942431

In [11]:
df1.printSchema()

root
 |-- LOGICALREF: string (nullable = true)
 |-- ITEMCODE: integer (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- AMOUNT: string (nullable = true)
 |-- PRICE: string (nullable = true)
 |-- LINENETTOTAL: string (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- STARTDATE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- LATITUDE: string (nullable = true)



In [13]:
df2 = df1 \
.withColumn("LOGICALREF", F.col("LOGICALREF").cast(IntegerType())) \
.withColumn("ITEMCODE", F.col("ITEMCODE").cast(IntegerType())) \
.withColumn("AMOUNT", F.regexp_replace("AMOUNT", ",", ".").cast(FloatType())) \
.withColumn("PRICE", F.regexp_replace("PRICE", ",", ".").cast(FloatType())) \
.withColumn("LINENETTOTAL", F.regexp_replace("LINENETTOTAL", ",", ".").cast(FloatType())) \
.withColumn("STARTDATE",  F.unix_timestamp( F.to_timestamp(F.col("STARTDATE"), "d.MM.yyyy HH:mm")) * 1000 ) \
.withColumn("LONGITUDE", F.regexp_replace("LONGITUDE", ",", ".").substr(1, 6).cast(FloatType())) \
.withColumn("LATITUDE", F.regexp_replace("LATITUDE", ",", ".").substr(1, 6).cast(FloatType())) \
.withColumn("LOCATION", F.array(  F.col("LONGITUDE"), F.col("LATITUDE"))) \
                         .filter("LONGITUDE < 100") # filter wrong longitude

In [15]:
df2.limit(5).toPandas()

,LOGICALREF,ITEMCODE,ITEMNAME,AMOUNT,PRICE,LINENETTOTAL,BRANCH,CITY,BRAND,STARTDATE,LONGITUDE,LATITUDE,LOCATION
0,2,20868,KIRMIZI MERCIMEK,1.006,2.80,2.82,Batman Subesi,Batman,BAKLİYAT,1483424700000,41.134998,37.881001,"[41.1349983215332, 37.88100051879883]"
1,3,8583,"TEST MATIK 1,5 KG NORMAL",1.000,4.95,4.95,Batman Subesi,Batman,TEST,1483424700000,41.134998,37.881001,"[41.1349983215332, 37.88100051879883]"
2,4,1454,BIZIM MAKARNA BONCUK,1.000,1.10,1.10,Batman Subesi,Batman,ÜLKER,1483424700000,41.134998,37.881001,"[41.1349983215332, 37.88100051879883]"
3,5,13519,FILIZ MAKARNA KISA KESME 500 GR,1.000,1.10,1.10,Batman Subesi,Batman,FİLİZ,1483424700000,41.134998,37.881001,"[41.1349983215332, 37.88100051879883]"
4,6,8639,BINGO ULTRA CAM.SUYU 750 ML KLASIK,1.000,2.45,2.45,Batman Subesi,Batman,BİNGO,1483424700000,41.134998,37.881001,"[41.1349983215332, 37.88100051879883]"


In [16]:
df2.printSchema()

root
 |-- LOGICALREF: integer (nullable = true)
 |-- ITEMCODE: integer (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- AMOUNT: float (nullable = true)
 |-- PRICE: float (nullable = true)
 |-- LINENETTOTAL: float (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- STARTDATE: long (nullable = true)
 |-- LONGITUDE: float (nullable = true)
 |-- LATITUDE: float (nullable = true)
 |-- LOCATION: array (nullable = false)
 |    |-- element: float (containsNull = true)



### 4. Elasticsearch

In [17]:
market_index =  {
  "settings": {
    "index": {
      "analysis": {
        "analyzer": {
          "custom_analyzer":
          {
            "type":"custom",
            "tokenizer":"standard",
            "filter":[
              "lowercase", "custom_edge_ngram"
            ]
          }
        },
        "filter": {
          "custom_edge_ngram": {
            "type": "edge_ngram",
            "min_gram":2,
            "max_gram": 10
            }
          }
        }
      }
    },
    "mappings": {
    "properties": {
      "LOGICALREF":  {"type": "integer"},  
      "ITEMCODE":    {"type": "integer" }, 
      "ITEMNAME":    {"type": "text"},
      "AMOUNT":      {"type": "float"},
      "PRICE":       {"type": "float"},
      "LINENETTOTAL":{"type": "float"},
      "BRANCH":      {"type": "keyword"},
      "CITY":        {"type": "keyword"},
      "BRAND":       {"type": "keyword"},
      "STARTDATE":   {"type": "date"},
      "LOCATION":    {"type": "geo_point"}
        
    }
  }
  }

In [18]:
es = Elasticsearch()

In [19]:
try:
    es.indices.delete("market_index")
    print("market_index index deleted.")
except:
    print("No index")

market_index index deleted.


In [21]:
es.indices.create("market_index", body=market_index)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'market_index'}

### 5. Write to Elasticsearch

In [22]:
start_time = time.time()

df2.drop("LONGITUDE","LATITUDE").write \
    .format("org.elasticsearch.spark.sql") \
    .mode("overwrite") \
    .option("es.nodes", "localhost") \
    .option("es.port","9200") \
    .save("market_index")


print("----- %s secs -----" %(time.time() - start_time))

2023-04-25 09:23:43,773 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


----- 59.714521408081055 secs -----


### 6. Read to Elasticsearch

In [35]:
df_elastic = spark.read \
                  .format("org.elasticsearch.spark.sql") \
                  .option("es.nodes", "localhost") \
                  .option("es.port","9200") \
                  .load("market_index")

In [36]:
df_elastic.limit(5).toPandas()

,AMOUNT,BRANCH,BRAND,CITY,ITEMCODE,ITEMNAME,LINENETTOTAL,LOCATION,LOGICALREF,PRICE,STARTDATE
0,1.635,İzmir Subesi,HAL,İzmir,5705,DOLMALIK BIBER,9.16,"[27.128, 38.418]",500465,5.60,2017-03-17 20:42:00
1,0.690,İzmir Subesi,HAL,İzmir,5694,DOMATES,1.31,"[27.128, 38.418]",500466,1.90,2017-03-17 20:42:00
2,1.000,İstanbul Subesi,ÜLKER,İstanbul,20512,CAFE CROWN 2 U 1 FINDIKLI,0.35,"[28.977, 41.005]",23077,0.35,2017-01-09 16:03:00
3,1.000,İstanbul Subesi,ÜLKER,İstanbul,20512,CAFE CROWN 2 U 1 FINDIKLI,0.35,"[28.977, 41.005]",23078,0.35,2017-01-09 16:03:00
4,1.575,İzmir Subesi,HAL,İzmir,5699,KABAK,3.07,"[27.128, 38.418]",500468,1.95,2017-03-17 20:42:00


In [26]:
df_elastic.selectExpr("MIN(STARTDATE)","MAX(STARTDATE)").show()

+-------------------+-------------------+
|     min(STARTDATE)|     max(STARTDATE)|
+-------------------+-------------------+
|2017-01-03 09:25:00|2017-05-25 21:00:00|
+-------------------+-------------------+



In [37]:
df_elastic.printSchema()

root
 |-- AMOUNT: float (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- ITEMCODE: integer (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- LINENETTOTAL: float (nullable = true)
 |-- LOCATION: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- LOGICALREF: integer (nullable = true)
 |-- PRICE: float (nullable = true)
 |-- STARTDATE: timestamp (nullable = true)



In [38]:
df_elastic.count()

936156